In [1]:
library(tidyverse)
library(data.table)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [2]:
df_merged <- fread("../input/calvin-smith-data-clean/df_merged.csv",
                   drop = c('V1', 'angleToTarget', 'diff_wr', 'team', 'possessionTeam'))

colnames(df_merged)
head(df_merged)

[1] "gameId"              "playId"              "passResult"         
 [4] "targetNflId"         "x"                   "y"                  
 [7] "o"                   "dir"                 "s"                  
[10] "a"                   "event"               "epa"                
[13] "frameId"             "nflId"               "displayName"        
[16] "position"            "route"               "isDefensivePI"      
[19] "targetedReceiver"    "team_name"           "ball_x"             
[22] "ball_y"              "ball_snap_x"         "max_space_available"
[25] "target_x"            "target_y"            "qb_x"               
[28] "qb_y"                "qb_s"                "qb_a"               
[31] "distanceFromBall"    "distanceFromTarget"  "distance_from_qb"

gameId,playId,passResult,targetNflId,x,y,o,dir,s,a,⋯,max_space_available,target_x,target_y,qb_x,qb_y,qb_s,qb_a,distanceFromBall,distanceFromTarget,distance_from_qb
<int>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018090600,75,C,2495454,31.11,16.83333,104.49,36.45,0.01,0.01,⋯,90.11,28.64,9.193333,28.26,26.66333,0.00,0.00,9.726813,8.029352,10.23481
2018090600,75,C,2495454,31.11,16.82333,104.49,37.08,0.01,0.01,⋯,90.11,28.65,9.193333,28.24,26.65333,0.00,0.00,9.736735,8.016764,10.24040
2018090600,75,C,2495454,31.11,16.82333,102.44,34.00,0.00,0.01,⋯,90.11,28.68,9.193333,28.22,26.65333,0.03,0.82,9.762797,8.007609,10.24602
2018090600,75,C,2495454,31.11,16.83333,99.80,278.87,0.01,0.18,⋯,90.11,28.72,9.193333,28.21,26.65333,0.22,2.24,9.814622,8.005105,10.23926
2018090600,75,C,2495454,31.12,16.82333,99.07,288.04,0.07,0.41,⋯,90.11,28.76,9.203333,28.16,26.65333,0.61,3.46,9.884452,7.977092,10.26599
2018090600,75,C,2495454,31.13,16.81333,98.38,317.54,0.14,0.61,⋯,90.11,28.84,9.213333,28.06,26.66333,1.18,4.58,9.927724,7.937512,10.31733


In [3]:
# load in data from summarise_plays notebook
nearest_stats <- fread("../input/summarise-plays/nearest_stats.csv",
                   drop = c('V1'))


nrow(nearest_stats)
head(nearest_stats)

[1] 3468218

gameId,playId,frameId,off_nflId,nflId,displayName,dist_from_off,nearest_is_target,distanceFromBall,off_distanceFromBall,ball_behind_db,diff_in_speed,diff_in_acceleration,diff_in_dir,receiver_behind_db,receiver_distance_from_qb,num_of_receivers_nearest,dist_rate_of_change,his_off_most_open
<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>
2018090600,75,11,2533040,79848,Malcolm Jenkins,1.920339,0,9.726813,9.334077,0,0.01,0.01,258.48,1.89,9.538433,2,0.000000000,0
2018090600,75,12,2533040,79848,Malcolm Jenkins,1.922134,0,9.736735,9.334077,0,0.01,0.01,276.29,1.89,9.530519,2,0.000000000,0
2018090600,75,13,2533040,79848,Malcolm Jenkins,1.912302,0,9.762797,9.365452,0,-0.02,-0.29,280.46,1.88,9.533651,2,0.000000000,0
2018090600,75,14,2533040,79848,Malcolm Jenkins,1.908743,0,9.814622,9.353614,0,-0.07,-0.50,34.84,1.88,9.544658,2,-0.006038243,0
2018090600,75,15,2533040,79848,Malcolm Jenkins,1.889047,0,9.884452,9.351048,0,-0.22,-1.20,9.95,1.86,9.563472,2,-0.017213601,0
2018090600,75,16,2533040,79848,Malcolm Jenkins,1.859516,0,9.927724,9.341333,0,-0.40,-1.57,29.29,1.83,9.600417,2,-0.027603487,0


In [4]:
df_merged <- left_join(nearest_stats,
                       df_merged,
                       by=c('gameId', 'playId', 'frameId', 'nflId', 'displayName'))

head(df_merged)

gameId,playId,frameId,off_nflId,nflId,displayName,dist_from_off,nearest_is_target,distanceFromBall.x,off_distanceFromBall,⋯,max_space_available,target_x,target_y,qb_x,qb_y,qb_s,qb_a,distanceFromBall.y,distanceFromTarget,distance_from_qb
<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018090600,75,11,2533040,79848,Malcolm Jenkins,1.920339,0,9.726813,9.334077,⋯,90.11,28.64,9.193333,28.26,26.66333,0.00,0.00,9.726813,8.029352,10.23481
2018090600,75,12,2533040,79848,Malcolm Jenkins,1.922134,0,9.736735,9.334077,⋯,90.11,28.65,9.193333,28.24,26.65333,0.00,0.00,9.736735,8.016764,10.24040
2018090600,75,13,2533040,79848,Malcolm Jenkins,1.912302,0,9.762797,9.365452,⋯,90.11,28.68,9.193333,28.22,26.65333,0.03,0.82,9.762797,8.007609,10.24602
2018090600,75,14,2533040,79848,Malcolm Jenkins,1.908743,0,9.814622,9.353614,⋯,90.11,28.72,9.193333,28.21,26.65333,0.22,2.24,9.814622,8.005105,10.23926
2018090600,75,15,2533040,79848,Malcolm Jenkins,1.889047,0,9.884452,9.351048,⋯,90.11,28.76,9.203333,28.16,26.65333,0.61,3.46,9.884452,7.977092,10.26599
2018090600,75,16,2533040,79848,Malcolm Jenkins,1.859516,0,9.927724,9.341333,⋯,90.11,28.84,9.213333,28.06,26.66333,1.18,4.58,9.927724,7.937512,10.31733


In [5]:
write.csv(df_merged, 'full_tracking_df.csv')